# Implementing the Transformer Decoder in Keras

Here again what the decoder does is that it takes the input (target) sequence's "raw" embeddings and, through multiple self-attention + cross-attention layers, gradually _transforms_ them to projections that are more representative and meaningful based on overall context. This is, of course, done in parallel over all tokens all at once. (At inference we generate one token at a time, but we'll get to inference in future chapters).

In [1]:
from numpy import random
from tensorflow import shape
from tensorflow.keras.layers import Dropout, Layer

from xformer.common import AddAndNorm, FeedForward
from xformer.multihead_attention import MultiHeadAttention
from xformer.positional_encoding import CustomEmbeddingWithFixedPosnWts

## 18.1 Recap of the Transformer Decoder

In NLP, typically a sequence-to-sequence model such as translation would have be an encoder+decoder transformer. Once again, the architecture of the decoder has _a lot_ in common with the encoder. Let's focus on its few differences and important details.  

It has _three_ sub-layers instead of two:
1. One multi-head self-attention with queries, keys and values coming from the embedded and positionally-encoded input sequences. This is architecturally identical to the multi-head self-attention layer in the encoder. (Just remember that the inputs come from _target_ sentences).
2. It has an additional multi-head attention sub-layer which is _not_ self-attending. It gets its keys and values coming from the output of the transformer's encoder and it gets is queries from its own multi-head self-attention sub-layer (#1 above). We can call this one multi-head _cross_-attention if you want.
3. As with the encoder, it has a fully-connected feed-forward sub-layer after that.  

As with the encoder, each sub-layer above is followed by an "Add-and-Norm" layer normalization sub-layer. And just as before, regularization is performed by applying a dropout layer to the outputs of each of the above 3 sub-layers right before the normalization step, as well as to the positionally-encoded embeddings right before they are fed into the decoder.

## 18.2 Implementing the Transformer Decoder from Scratch

Let's just jump right into implementing it, starting by defining the `DecoderLayer` and `Decoder` classes, simililarly to how we did things for the encoder and reusing a lot of the code from before.

In [2]:
# Implementing the Decoder Layer
class DecoderLayer(Layer):
    def __init__(self, n_heads, d_model, d_ff, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.multihead_attention1 = MultiHeadAttention(n_heads, d_model)
        self.dropout1 = Dropout(dropout_rate)
        self.add_norm1 = AddAndNorm()
        self.multihead_attention2 = MultiHeadAttention(n_heads, d_model)
        self.dropout2 = Dropout(dropout_rate)
        self.add_norm2 = AddAndNorm()
        self.feed_forward = FeedForward(d_ff, d_model)
        self.dropout3 = Dropout(dropout_rate)
        self.add_norm3 = AddAndNorm()

    def call(self, x, encoder_output, lookahead_mask, padding_mask, training):
        # Multi-head self-attention layer
        multihead_output1 = self.multihead_attention1(x, x, x, lookahead_mask)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Add in a dropout layer
        multihead_output1 = self.dropout1(multihead_output1, training=training)

        # Followed by an Add & Norm layer
        addnorm_output1 = self.add_norm1(x, multihead_output1)
        # Expected output shape = (batch_size, sequence_length, d_model)

        # Followed by another multi-head (cross-)attention layer
        multihead_output2 = self.multihead_attention2(
            addnorm_output1, encoder_output, encoder_output, padding_mask
        )

        # Add in another dropout layer
        multihead_output2 = self.dropout2(multihead_output2, training=training)
        
        # Followed by another Add & Norm layer
        addnorm_output2 = self.add_norm2(addnorm_output1, multihead_output2)
        
        # Followed by a fully connected layer
        feedforward_output = self.feed_forward(addnorm_output2)
        # Expected output shape = (batch_size, sequence_length, d_model)
        # Add in another dropout layer
        feedforward_output = self.dropout3(
            feedforward_output, training=training
        )
        # Followed by another Add & Norm layer
        return self.add_norm3(addnorm_output2, feedforward_output)

In [3]:
# Implementing the Decoder
class Decoder(Layer):
    def __init__(
        self,
        vocab_size,
        sequence_length,
        n_heads,
        d_model,
        d_ff,
        n,
        dropout_rate,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.pos_encoding = CustomEmbeddingWithFixedPosnWts(
            sequence_length, vocab_size, d_model
        )
        self.dropout = Dropout(dropout_rate)
        self.decoder_layer = [
            DecoderLayer(n_heads, d_model, d_ff, dropout_rate) for _ in range(n)
        ]

    def call(
        self,
        output_target,
        encoder_output,
        lookahead_mask,
        padding_mask,
        training,
    ):
        # Generate the positional encoding
        pos_encoding_output = self.pos_encoding(output_target)
        # Expected output shape = (number of sentences, sequence_length, d_model)
        # Add in a dropout layer
        x = self.dropout(pos_encoding_output, training=training)
        # Pass on the positional encoded values to each encoder layer
        for i, layer in enumerate(self.decoder_layer):
            x = layer(x, encoder_output, lookahead_mask, padding_mask, training)
        return x

## 18.3 Testing Out the Code

As before, let's test it out with parameter values from AIAYN. We'll use dummy data for the target sequences _and_ for our encoder output. Also we won't be using masks yet.

In [4]:
h = 8  # Number of self-attention heads
d_ff = 2048  # Dimensionality of the inner fully-connected layer
d_model = 512  # Dimensionality of the model
n = 6  # Number of layers in the encoder stack

batch_size = 64  # Batch size from the training process
dropout_rate = 0.1  # Frequency of dropping the input units in dropout layers

dec_vocab_size = 20  # Vocabulary size for the decoder
input_seq_length = 5  # Maximum length of the input sequence
input_seq = random.random((batch_size, input_seq_length))
enc_output = random.random((batch_size, input_seq_length, d_model))

decoder = Decoder(
    dec_vocab_size, input_seq_length, h, d_model, d_ff, n, dropout_rate
)

print(decoder(input_seq, enc_output, None, None, True))

tf.Tensor(
[[[-1.091551    0.5167615   1.5189791  ...  0.13476753 -0.04141628
    0.53338975]
  [-1.6567624   0.20429553  1.2669424  ... -0.19116628  0.27740058
    0.4424816 ]
  [-0.19240509  0.4964267   2.168655   ...  0.30416283  0.63593733
    0.34734112]
  [-0.84042555  0.07570388  1.7701573  ...  0.08968826  0.04672641
    1.1327897 ]
  [-1.300864    0.34656417  1.8693477  ... -0.6912628   0.24487172
    0.01345476]]

 [[-1.5962428  -0.01789353  1.4801825  ...  0.1891943   0.00726201
    0.27039957]
  [-1.1815863  -0.04690985  1.94125    ...  0.11562945 -0.2635349
   -0.24138254]
  [-1.2886317  -0.4856953   1.5456611  ...  0.06169762 -0.5280251
   -0.12688605]
  [-1.5507399  -0.04944961  1.8819788  ...  0.59754694 -0.28177863
   -0.8248383 ]
  [-1.7740687  -0.96497697  1.2581016  ... -0.24477297 -0.48495072
   -0.04392153]]

 [[-1.670989    0.40551654  1.5721772  ... -0.3012978  -0.11557302
   -0.74379003]
  [-1.5104159   0.6682517   1.7245147  ... -0.29477245 -0.04250039
    0.5